# Turbine check-out/calibration notebook

Use this notebook to set up a series of runs for turbine calibration and check-out

In [1]:
%%capture 

# Important header information
amrwindfedir = '/projects/wind_uq/lcheung/amrwind-frontend'
import sys, os
sys.path.insert(1, amrwindfedir)

# Load the libraries
import matplotlib.pyplot    as plt
import amrwind_frontend as amrwind
import tempfile
import shutil
# Also ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Make all plots inline 
%matplotlib inline

**Note**: Make sure there is a turbine file which looks like:

In [ ]:
with open(amrwindfedir+"/turbines/nrel28-127_calib.yaml")as f: print(f.read())

Note that both `Actuator_epsilon` and `Actuator_epsilon_tower` are commented out.  We can set those values later.

In [3]:
# Start the amrwind_frontend app 
case = amrwind.MyApp.init_nogui()

In [ ]:
curdir=os.getcwd()

# Create a temporary working directory and switch to it

# Uncomment these lines to make your own tempdir
## tempdir = 'tempturbinedir'
## os.mkdir(tempdir)
# Otherwise use a system temporary directory
tempdir = tempfile.mkdtemp(prefix='tempturbine-')
os.chdir(tempdir)
print('Using temporary directory: '+tempdir)

## Set up the basic simulation properties

In [5]:
rhodensity = 1.225
WS         = 11.0                                     # This is temporary, we will vary across a range below
eps        = 5.0
case.setAMRWindInput('useWSDir', True)
case.setAMRWindInput('ABL_windspeed', WS, forcechange=True)
case.setAMRWindInput('ABL_winddir', 270,  forcechange=True)
case.ABL_calculateWindVector()

In [6]:
case.setAMRWindInput('time_control',   ['const dt'])
case.setAMRWindInput('time.stop_time', 360)
case.setAMRWindInput('time.fixed_dt',  0.1)
case.setAMRWindInput('incflo.physics', ['FreeStream', 'Actuator'])

In [7]:
case.setAMRWindInput('ConstValue.density.value',  rhodensity)
case.setAMRWindInput('incflo.density',            rhodensity)
case.setAMRWindInput('incflo.velocity',           [WS, 0.0, 0.0])
case.setAMRWindInput('ConstValue.velocity.value', [WS, 0.0, 0.0])
case.setAMRWindInput('turbulence.model',          ['Laminar'])

## Set the BC's

In [8]:
case.setAMRWindInput('is_periodicx', False)
case.setAMRWindInput('is_periodicy', True)
case.setAMRWindInput('is_periodicz', True)  

case.setAMRWindInput('xlo.type', 'mass_inflow')
case.setAMRWindInput('xhi.type', 'pressure_outflow')
case.setAMRWindInput('xlo.density',  rhodensity)
case.setAMRWindInput('xlo.velocity', [WS, 0.0, 0.0])

In [9]:
# Add the turbine specifications flag
## Note that the options field can have things like:
##    ADparam_TwrAero:False ADparam_TwrShadow:0 FSTparam_TMax:181234.0
## To specify changes to the OpenFAST model
turbinescsv="""
# CSV file should have columns with
# name, x, y, type, yaw, hubheight, options
T0,     0, 0, NREL 2.8-127 ADM bugfix, 270.0, 0.0, ADparam_TwrAero:True ADparam_TwrShadow:0
"""
case.setAMRWindInput('turbines_csvtextbox',  turbinescsv)

In [10]:
case.setAMRWindInput('turbines_domainsize', [2560.0*2, 960.0*2, 960.0*2])
case.setAMRWindInput('turbines_backgroundmeshsize', 20.0)
case.setAMRWindInput('turbines_deleteprev', True)
case.setAMRWindInput('turbines_freespace', True)   # Make this have no ground
#case.setAMRWindInput('zlo_type', 'slip_wall')
#case.setAMRWindInput('zhi_type', 'slip_wall')

In [ ]:
# Preview the turbine layout
fig, ax = plt.subplots(figsize=(5,5), facecolor='w', dpi=150)
case.turbines_previewAllTurbines(ax=ax)

In [ ]:
# Now create the turbines
case.turbines_createAllTurbines()

# Print out existing list of turbines, just to confirm that the turbines got made
print(case.listboxpopupwindict['listboxactuator'].getitemlist())

In [13]:
# Set some globals
case.setAMRWindInput('Actuator_default_type', ['TurbineFastDisk'])
case.setAMRWindInput('Actuator_TurbineFastDisk_density',       rhodensity,      forcechange=True)
case.setAMRWindInput('Actuator_TurbineFastDisk_epsilon',       [eps, eps, eps], forcechange=True)  # You can change this
case.setAMRWindInput('Actuator_TurbineFastDisk_epsilon_tower', [eps, eps, eps], forcechange=True)  # You can change this

## Add turbine refinement

In [14]:
refinementcsv="""
# CSV file should have columns with
# level, upstream, downstream, lateral, below, above, options
level, upstream, downstream, lateral, below, above, options
0,     10,       10,         3,       2,     2,     center:farm
1,     2.5,      2.5,        2.5,     1.2,   1.2,   center:farm
2,     1,        1,          0.75,    0.8,   0.8,  
"""
case.setAMRWindInput('refine_csvtextbox', refinementcsv)
case.setAMRWindInput('refine_deleteprev', True)

In [ ]:
case.refine_createAllZones()
# Print out existing list of turbines
print(case.listboxpopupwindict['listboxtagging'].getitemlist())

In [ ]:
# Plot the domain
fig, ax2 = plt.subplots(figsize=(5,5), facecolor='w', dpi=125)
case.popup_storteddata['plotdomain']['plot_turbines']        = case.listboxpopupwindict['listboxactuator'].getitemlist()
case.popup_storteddata['plotdomain']['plot_refineboxes']     = case.listboxpopupwindict['listboxtagging'].getitemlist()
case.plotDomain(ax=ax2)

In [ ]:
# Plot the domain
fig, ax = plt.subplots(figsize=(8,6), facecolor='w', dpi=125)
case.popup_storteddata['plotdomain']['plot_chooseview']      = 'YZ'
case.popup_storteddata['plotdomain']['plot_refineboxes']     = case.listboxpopupwindict['listboxtagging'].getitemlist()
case.popup_storteddata['plotdomain']['plot_sampleprobes']    = [] #case.listboxpopupwindict['listboxsampling'].getitemlist()
case.popup_storteddata['plotdomain']['plot_turbines']        = case.listboxpopupwindict['listboxactuator'].getitemlist()
case.plotDomain(ax=ax)

In [ ]:
case.estimateMeshSize()

### Create sampling planes

In [19]:
samplingcsv="""
# CSV file should have columns withturbinescsv=
# name, type, upstream, downstream, lateral, below, above, n1, n2, options
name, type, upstream, downstream, lateral, below, above, n1, n2, options
#cl1, centerline, 1,  0, none, none,  none,  11, 11, none
#rp1, rotorplane, 2,  0, none, none,  none,  11, 11, none
#sw1, streamwise, 2,  1, 1, 0.5,  0.5,  11, 11, usedx:0.25 noffsets:1
hh,  hubheight,2560,2560,960, 0,  none,  11, 11, usedx:10 units:meter center:farm orientation:x
"""
case.setAMRWindInput('sampling_csvtextbox', samplingcsv)
case.setAMRWindInput('sampling_deleteprev', True)

In [ ]:
case.sampling_createAllProbes(verbose=False)
# Print out existing list of turbines
print(case.listboxpopupwindict['listboxsampling'].getitemlist())

In [ ]:
# Plot the domain
fig, ax = plt.subplots(figsize=(8,6), facecolor='w', dpi=125)
case.popup_storteddata['plotdomain']['plot_refineboxes']     = [] #case.listboxpopupwindict['listboxtagging'].getitemlist()
case.popup_storteddata['plotdomain']['plot_sampleprobes']    = case.listboxpopupwindict['listboxsampling'].getitemlist()
case.popup_storteddata['plotdomain']['plot_turbines']        = case.listboxpopupwindict['listboxactuator'].getitemlist()
case.plotDomain(ax=ax)

In [ ]:
print(case.writeAMRWindInput(''))

In [23]:
# Clean up the temporary directory
docleanup = True
os.chdir(curdir)
if docleanup:
    try:
        shutil.rmtree(tempdir)
    except:
        print("Error cleaning up: "+tempdir)

## Set up a wind sweep

In [24]:
# Set the parameters
case.setAMRWindInput('sweep_windspeeds', '7 9 10 11')
case.setAMRWindInput('sweep_winddirs',   '270')
case.setAMRWindInput('sweep_caseprefix', 'Calibrate_dx2p5_{CASENUM}_EPS%0.2f_WS_{WS}'%eps)
case.setAMRWindInput('sweep_dirprefix',  'Calibrate_dx2p5_{CASENUM}_EPS%0.2f_WS_{WS}'%eps)
case.setAMRWindInput('sweep_usenewdirs', True)
case.setAMRWindInput('sweep_logfile',    'RunCalibration_dx2.5_Eps%0.2f_logfile.yaml'%eps)

In [25]:
# Submission parameters
case.popup_storteddata['submitscript']['submitscript_numnodes'] = 8
case.popup_storteddata['submitscript']['submitscript_runtime']  = '4:00:00'
case.popup_storteddata['submitscript']['submitscript_wcid']     = 'fy210193'
case.popup_storteddata['submitscript']['submitscript_partition']= 'batch,short'
case.popup_storteddata['submitscript']['submitscript_corespernode'] = 16
case.popup_storteddata['submitscript']['submitscript_queue']    = 'normal'
case.popup_storteddata['submitscript']['submitscript_jobname']  = 'Cstm2p8'
case.popup_storteddata['submitscript']['submitscript_exe']      = '/projects/wind_uq/lcheung/AMRWindBuilds/hfm.20240102/amr-wind/build/amr_wind'

case.setAMRWindInput('sweep_createsubmitscript', True)
case.setAMRWindInput('sweep_submitjob',          False)

In [ ]:
# Set up the cases
case.sweep_SetupRunParamSweep(verbose=True)

### Save the calibration

In [ ]:
# Save the wind farm setup
case.setAMRWindInput('farm_usercomments', 'Test calibration process.')
case.writeFarmSetupYAML('TurbineCalibrateCustom2p8_dx2.5_Eps%0.2f.yaml'%eps)